In [1]:
import pandas as pd
import os

In [2]:
os.chdir("../Helper_Functions")
from evaluation_pipeline import calculate_feature_gain, get_final_eval
# Needs to be current working directory
os.chdir("../Main")

/home/marc/anaconda3/envs/deeplearning2/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
item_orders_dec = pd.read_csv("../Data/train_before_dec.csv")
item_orders_dec["date"] = pd.to_datetime(item_orders_dec["date"])
item_orders_dec

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,purchaseWeek
0,2020-06-01,38769,3477,1,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]",1
1,2020-06-01,42535,30474,1,193,10,3,229,3,132,"[3459, 3738, 679, 1628, 4072]",1
2,2020-06-01,42535,15833,1,1318,4,1,455,0,108,"[2973, 2907, 2749, 3357]",1
3,2020-06-01,42535,20131,1,347,4,0,291,3,44,"[30, 1515, 1760, 2932, 1287, 2615, 3727, 2450,...",1
4,2020-06-01,42535,4325,1,539,6,0,303,0,45,"[3104, 1772, 2029, 1274, 3915, 888, 1118, 3882...",1
...,...,...,...,...,...,...,...,...,...,...,...,...
767126,2020-11-30,22541,746,1,406,10,0,503,0,17,"[3704, 2833, 146, 308, 3540, 2980, 1138]",0
767127,2020-11-30,36315,22834,20,993,10,3,515,0,190,[3916],0
767128,2020-11-30,7736,18630,3,6,4,0,291,0,44,"[2977, 30, 1515, 1760, 2932, 377, 3727, 747, 3...",0
767129,2020-11-30,7736,29657,8,18,10,0,345,0,84,"[1515, 2330, 1760, 3245, 2443, 3173, 3915, 391...",0


In [4]:
item_orders_jan = pd.read_csv("../Data/train_before_jan.csv")
item_orders_jan["date"] = pd.to_datetime(item_orders_jan["date"])
item_orders_jan

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,categories,purchaseWeek
0,2020-06-01,38769,3477,1,186,6,0,196,0,45,"[74, 4109, 3867, 803, 4053]",1
1,2020-06-01,42535,30474,1,193,10,3,229,3,132,"[3459, 3738, 679, 1628, 4072]",1
2,2020-06-01,42535,15833,1,1318,4,1,455,0,108,"[2973, 2907, 2749, 3357]",1
3,2020-06-01,42535,20131,1,347,4,0,291,3,44,"[30, 1515, 1760, 2932, 1287, 2615, 3727, 2450,...",1
4,2020-06-01,42535,4325,1,539,6,0,303,0,45,"[3104, 1772, 2029, 1274, 3915, 888, 1118, 3882...",1
...,...,...,...,...,...,...,...,...,...,...,...,...
905876,2020-12-31,38399,32075,1,1111,4,0,25,3,45,"[1353, 545, 2373, 4196]",0
905877,2020-12-31,38399,23050,1,1411,4,0,22,0,151,"[545, 1763, 3912, 3300, 3586, 3914, 3915, 3962...",0
905878,2020-12-31,21613,25422,1,539,10,0,345,0,99,"[3245, 1182]",0
905879,2020-12-31,27030,52,1,799,10,1,358,0,99,[1403],0


In [5]:
rf_dec = pd.read_csv("../Data/Feature CSVs/rfm_analysis_dec.csv")
rf_dec

,userID,recency,frequency,quantity_avg,quantity_total,R,F,RF_Segment_Concat,RF_Score,RF_Level
0,0,11,9,1.000000,9,3,2,32,5,2
1,1,2,10,1.300000,13,4,2,42,6,2
2,2,3,18,1.166667,21,4,3,43,7,3
3,3,10,31,1.290323,40,3,4,34,7,3
4,4,2,17,1.705882,29,4,3,43,7,3
...,...,...,...,...,...,...,...,...,...,...
45760,46133,9,21,1.095238,23,3,3,33,6,2
45761,46134,17,26,1.230769,32,3,4,34,7,3
45762,46135,18,6,1.666667,10,3,1,31,4,1
45763,46136,33,32,1.812500,58,2,4,24,6,2


In [6]:
rf_jan = pd.read_csv("../Data/Feature CSVs/rfm_analysis_jan.csv")
rf_jan

,userID,recency,frequency,quantity_avg,quantity_total,R,F,RF_Segment_Concat,RF_Score,RF_Level
0,0,21,14,1.000000,14,3,2,32,5,2
1,1,23,12,1.250000,15,2,2,22,4,1
2,2,13,20,1.200000,24,3,3,33,6,2
3,3,3,38,1.289474,49,4,4,44,8,3
4,4,33,17,1.705882,29,2,2,22,4,1
...,...,...,...,...,...,...,...,...,...,...
46026,46133,40,21,1.095238,23,1,3,13,4,1
46027,46134,19,27,1.222222,33,3,4,34,7,3
46028,46135,25,7,1.857143,13,2,1,21,3,1
46029,46136,4,35,1.971429,69,4,4,44,8,3


In [7]:
item_orders_dec_2 = pd.merge(item_orders_dec, rf_dec, on=["userID"], how="left")
item_orders_dec_2

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,...,purchaseWeek,recency,frequency,quantity_avg,quantity_total,R,F,RF_Segment_Concat,RF_Score,RF_Level
0,2020-06-01,38769,3477,1,186,6,0,196,0,45,...,1,1,66,1.454545,96,4,4,44,8,3
1,2020-06-01,42535,30474,1,193,10,3,229,3,132,...,1,71,18,1.111111,20,1,3,13,4,1
2,2020-06-01,42535,15833,1,1318,4,1,455,0,108,...,1,71,18,1.111111,20,1,3,13,4,1
3,2020-06-01,42535,20131,1,347,4,0,291,3,44,...,1,71,18,1.111111,20,1,3,13,4,1
4,2020-06-01,42535,4325,1,539,6,0,303,0,45,...,1,71,18,1.111111,20,1,3,13,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767126,2020-11-30,22541,746,1,406,10,0,503,0,17,...,0,1,8,1.250000,10,4,1,41,5,2
767127,2020-11-30,36315,22834,20,993,10,3,515,0,190,...,0,1,16,2.375000,38,4,3,43,7,3
767128,2020-11-30,7736,18630,3,6,4,0,291,0,44,...,0,1,13,2.076923,27,4,2,42,6,2
767129,2020-11-30,7736,29657,8,18,10,0,345,0,84,...,0,1,13,2.076923,27,4,2,42,6,2


In [8]:
item_orders_jan_2 = pd.merge(item_orders_jan, rf_jan, on=["userID"], how="left")
item_orders_jan_2

,date,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,...,purchaseWeek,recency,frequency,quantity_avg,quantity_total,R,F,RF_Segment_Concat,RF_Score,RF_Level
0,2020-06-01,38769,3477,1,186,6,0,196,0,45,...,1,26,73,1.410959,103,2,4,24,6,2
1,2020-06-01,42535,30474,1,193,10,3,229,3,132,...,1,9,23,1.086957,25,4,3,43,7,3
2,2020-06-01,42535,15833,1,1318,4,1,455,0,108,...,1,9,23,1.086957,25,4,3,43,7,3
3,2020-06-01,42535,20131,1,347,4,0,291,3,44,...,1,9,23,1.086957,25,4,3,43,7,3
4,2020-06-01,42535,4325,1,539,6,0,303,0,45,...,1,9,23,1.086957,25,4,3,43,7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905876,2020-12-31,38399,32075,1,1111,4,0,25,3,45,...,0,1,10,1.000000,10,4,1,41,5,2
905877,2020-12-31,38399,23050,1,1411,4,0,22,0,151,...,0,1,10,1.000000,10,4,1,41,5,2
905878,2020-12-31,21613,25422,1,539,10,0,345,0,99,...,0,1,21,1.000000,21,4,3,43,7,3
905879,2020-12-31,27030,52,1,799,10,1,358,0,99,...,0,1,14,1.214286,17,4,2,42,6,2


In [10]:
cols_to_drop = ["date", "categories", "purchaseWeek"] + ["quantity_avg", "quantity_total", "R", "F", "RF_Score","RF_Level"] 
# Add columns that you want to drop
#cols_to_drop = cols_to_drop + [] 
X_train_dec, y_train_dec = item_orders_dec_2.drop(cols_to_drop, axis=1), item_orders_dec_2["purchaseWeek"]
X_train_jan, y_train_jan = item_orders_jan_2.drop(cols_to_drop, axis=1), item_orders_jan_2["purchaseWeek"]

In [12]:
y_pred_dec, y_pred_jan, model = get_final_eval(X_train_dec, y_train_dec, X_train_jan, y_train_jan, path_to_data="../Data", model="rf", n_iter=1)

Currently fitting Random Forest Model
Final score is: 0.37


In [12]:
submission_dec = pd.read_csv("../Data/test_dec.csv", sep="|")
submission_jan = pd.read_csv("../Data/test_jan.csv", sep="|")

In [13]:
submission_dec["prediction"] = y_pred_dec
submission_dec.to_csv("../Data/submission_dec.csv")

submission_jan["prediction"] = y_pred_jan
submission_jan.to_csv("../Data/submission_jan.csv")

In [18]:
X_train_dec.head()

,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,recency,frequency,RF_Segment_Concat
450534,3812,3590,1,1371,4,0,487,3,44,8,47,44
118849,33872,3072,2,6,6,0,181,0,17,4,11,42
66934,13470,18625,1,839,4,0,92,3,144,1,44,44
646002,2933,6147,1,618,4,0,321,0,144,18,10,32
535130,31830,26503,1,618,4,0,71,0,53,9,24,34


In [12]:
user_items_feb = pd.read_csv("../Data/old/submission.csv", sep='|')
features_feb = pd.merge(user_items_feb, item_orders_jan, on=["userID", "itemID"], how="left").drop_duplicates(["userID", "itemID"])
rmf_feb = pd.read_csv("../Data/Feature CSVs/rfm_analysis_feb.csv")
features_feb = pd.merge(features_feb, rmf_feb, how='left', on=['userID'])
features_feb = features_feb.drop(columns=cols_to_drop, axis=1)
features_feb = features_feb.drop(columns=['prediction'])
features_feb.fillna(0, inplace=True)
display(features_feb.head())
predictions = model.predict(features_feb)
user_items_feb['prediction'] = predictions
user_items_feb.to_csv('../Data/final_submission.csv', sep='|', index=False, header=True)

,userID,itemID,order,brand,feature_1,feature_2,feature_3,feature_4,feature_5,recency,frequency,RF_Segment_Concat
0,0,20664,1.0,408.0,4.0,0.0,284.0,0.0,66.0,7,18,42
1,0,28231,1.0,193.0,4.0,3.0,468.0,3.0,108.0,7,18,42
2,13,2690,1.0,406.0,4.0,3.0,491.0,0.0,66.0,2,50,44
3,15,1299,1.0,1056.0,4.0,0.0,474.0,-1.0,108.0,7,49,44
4,15,20968,1.0,1315.0,4.0,0.0,444.0,0.0,144.0,7,49,44
